In [4]:
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1
import os
import sys
# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

import matplotlib.pyplot as plt
%matplotlib inline
from dotenv import load_dotenv
%aimport settings
import settings
import pandas as pd
import numpy as np

maxsize=3000
np.set_printoptions(threshold=sys.maxsize)
pd.options.display.max_columns = 999
pd.options.display.max_rows = 352905

import statsmodels.api as sm
from tqdm import trange
from tqdm import tqdm
import warnings
import math
from sklearn.preprocessing import PolynomialFeatures
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [6]:
sales=pd.read_csv(settings.raw_data+'/sales_train.csv')
item_cat=pd.read_csv(settings.raw_data+"/item_categories.csv")
item=pd.read_csv(settings.raw_data+"/items.csv")
sub=pd.read_csv(settings.raw_data+"/sample_submission.csv")
shops=pd.read_csv(settings.raw_data+"/shops.csv")
test=pd.read_csv(settings.raw_data+"/test.csv")

#### EDA

In [7]:
sales_item_price = sales[['item_id','item_price']].drop_duplicates(keep = 'first')

In [8]:
sales_item_price.shape

(120216, 2)

In [9]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [43]:
item_cat[item_cat.item_category_name.str.contains('PS')]

,item_category_name,item_category_id
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
5,Аксессуары - PSVita,5
10,Игровые консоли - PS2,10
11,Игровые консоли - PS3,11
12,Игровые консоли - PS4,12
13,Игровые консоли - PSP,13
14,Игровые консоли - PSVita,14


In [56]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [55]:
item.item_category_id.value_counts()

40    5035
55    2365
37    1780
31    1125
58     790
30     756
72     666
19     628
61     598
23     501
43     471
57     427
56     397
67     394
63     366
76     333
41     332
69     317
38     310
62     295
59     294
64     290
78     278
75     277
28     240
70     228
77     220
45     195
66     178
20     175
49     167
65     163
44     161
47     151
25     140
21     125
24     123
54      90
22      79
2       75
60      52
29      46
11      41
6       40
15      37
73      36
3       34
5       29
68      27
7       24
12      20
16      20
33      18
14      18
4       15
83      15
42      12
9       12
46      12
8        9
82       8
34       8
27       8
74       8
17       7
81       7
39       7
71       6
36       6
32       6
80       6
18       6
13       6
48       5
0        4
35       4
52       3
53       3
26       3
50       2
1        2
51       1
10       1
79       1
Name: item_category_id, dtype: int64

### merge dataframe

In [10]:
pd.options.display.max_rows = 999
sales['date'] = pd.to_datetime(sales['date'], format='%Y-%m-%d',infer_datetime_format=True)
sales_items = sales[['item_id','item_price']].drop_duplicates(keep='first')

In [11]:
sales_shop = pd.merge(sales, shops, on='shop_id', how='left')
df_train = pd.merge(sales_shop, pd.merge(item, item_cat, on='item_category_id', how='left'), on='item_id', how='left')

In [12]:
df_train.item_cnt_day.describe()
#sns.distplot(df_train_agg.item_cnt_day)

count    2.935849e+06
mean     1.242641e+00
std      2.618834e+00
min     -2.200000e+01
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.169000e+03
Name: item_cnt_day, dtype: float64

In [13]:
df_train = df_train[(df_train['item_cnt_day']>0) & (df_train['item_cnt_day']<df_train['item_cnt_day'].max())].copy()
df_train.sort_values(['date','shop_id','item_id'], inplace = True)

In [170]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
32569,01.01.2013,0,2,991,99.0,1.0,"Адыгея ТЦ ""Мега""","3D Action Puzzle ""Динозавры"" Тиранозавр",67,Подарки - Развитие
32552,01.01.2013,0,2,1472,2599.0,1.0,"Адыгея ТЦ ""Мега""","Assassin's Creed 3 [Xbox 360, русская версия]",23,Игры - XBOX 360
32632,01.01.2013,0,2,1905,249.0,1.0,"Адыгея ТЦ ""Мега""","Bestseller. Grand Theft Auto: San Andreas [PC,...",30,Игры PC - Стандартные издания
32459,01.01.2013,0,2,2920,599.0,2.0,"Адыгея ТЦ ""Мега""",Disney. LEGO Пираты Карибского моря (Essential...,21,Игры - PSP
32504,01.01.2013,0,2,3320,1999.0,1.0,"Адыгея ТЦ ""Мега""","FIFA 13 (с поддержкой PS Move) [PS3, русская в...",19,Игры - PS3


In [15]:
##Aggregate item count at monthly level
features = ['item_price']
col = 'item_cnt_day'
df_agg = df_train.groupby(['shop_id','item_id','date_block_num'])['item_cnt_day'].sum().reset_index(drop = False)
df_train_agg = pd.merge(df_agg, pd.merge(item, item_cat, on='item_category_id', how='left'), on='item_id', how='left')

for s in range(1, 5):
    df_train_agg["prev_shop{}_{}".format(col, s)] = df_train_agg.groupby(['date_block_num','shop_id','item_category_name'])[col].shift(s)
    features.append("prev_shop{}_{}".format(col, s))
df_train_agg.fillna(0, inplace = True)

### Outliers

In [16]:
df_train_agg = df_train_agg[df_train_agg['item_cnt_day']!=df_train_agg.item_cnt_day.max()].copy()##don't include outliers

In [17]:
df_train_agg.item_cnt_day.describe()
#sns.distplot(df_train_agg.item_cnt_day)

count    1.608224e+06
mean     2.269765e+00
std      8.269277e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      1.305000e+03
Name: item_cnt_day, dtype: float64

In [18]:
features

['item_price',
 'prev_shopitem_cnt_day_1',
 'prev_shopitem_cnt_day_2',
 'prev_shopitem_cnt_day_3',
 'prev_shopitem_cnt_day_4']

In [19]:
train = pd.merge(df_train_agg,sales_item_price, on='item_id',how='left')

In [21]:
X_train, X_test, y_train, y_test = train_test_split(train[features], train['item_cnt_day'], test_size=0.30, random_state=42)

In [22]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def Grid_Search_CV(X_train, y_train):

    estimator = RandomForestRegressor()
    param_grid = { 
            "n_estimators"  : [100,200,300, 500],
            "max_features"  : ["auto", "sqrt", "log2"],
            #"max_features" : [20, 30, 40],
            "max_depth"     : [3, 4, 6, None],
           'min_samples_leaf':[5, 8, 12],
           'min_samples_split':[8, 15, 25],
            "bootstrap": [True, False],
            }

    grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)

    grid.fit(X_train, y_train)

    return grid.best_score_ , grid.best_params_

def create_random_forest(train, test, train_labels, test_labels, best_params):
    rf = RandomForestRegressor(n_jobs=-1, random_state =50).set_params(**best_params)
    rf.fit(train,train_labels)
    pred = rf. predict(test)
    pred_train = rf. predict(train)
    
    x=np.median(train_labels)
    medianpredict1 = [x for _ in test_labels] 
    
    print('rmse baseline: ',rmse(medianpredict1, test_labels))
    print('MAE baseline: ',mean_absolute_error(test_labels,medianpredict1))
    print("rmse_test",rmse(pred, test_labels))
    print("MAE test", mean_absolute_error(test_labels, pred))
    print('rmse train', rmse(pred_train, train_labels))
    print('MAE train', mean_absolute_error(train_labels, pred_train))
    #tree_graph = tree.export_graphviz(rf, out_file=None, feature_names=features)
    return pred, pred_train, rf